# Descrizione notebook
In questo notebook si fa utilizzo dell'Item-Based Collaborative Filtering per raccomandare una barzelletta ad un utente.

In [1]:
# import delle librerie necessarie
import numpy as np
import pandas as pd
from itertools import combinations
import math

In [2]:
# lettura dei rating normalizzati dal relativo CSV
normalized_ratings_df = pd.read_csv('./data/normalized_ratings.csv')

# lettura dei rating non normalizzati dal relativo CSV
no_normalized_ratings_df = pd.read_csv('./data/no_normalized_ratings.csv')

# Similarità tra item: Adjusted Coseno Similarità

In [3]:
def cosin_similarity(x, y): 
    t1, t2, t3 = 0, 0, 0 
    for i, j in zip(x, y):
        t1+=i*j # numeratore
        t2+=i*i # denominatore_1
        t3+=j*j # denominatore_2
    return t1 / (np.sqrt(t2) * np.sqrt(t3))

# Predizione

In [4]:
def score_user_item(item_target, items_rated, active_user_rating, similarity_matrix):
    t1, t2 = 0, 0
    for i in items_rated:
        sim = float(similarity_matrix.loc[((similarity_matrix['elem_1'] == item_target) & (similarity_matrix['elem_2'] == i)) | ((similarity_matrix['elem_1'] == i) & (similarity_matrix['elem_2'] == item_target))]["similarity"])
        t1+= sim * float(active_user_rating[i]) # numeratore
        t2+= sim # denominatore
    score = t1 / t2
    return score

# Raccomandazione

In [5]:
# si costruisce la matrice di similarità, ovvero una lista di coppie di item, per 
# ogni coppia si specifica la similarità (ad -1 a 1)
all_jokes = (list(normalized_ratings_df.columns))[2:]
jokes_pairs = list(combinations(all_jokes, 2))

similarity_matrix = []

for i_1, i_2 in jokes_pairs:
    app = normalized_ratings_df[[i_1, i_2]].dropna()
    similarity_matrix.append([i_1, i_2, cosin_similarity(app[i_1], app[i_2])])
    
similarity_matrix = pd.DataFrame(similarity_matrix, columns = ['elem_1', 'elem_2', 'similarity'])
similarity_matrix.head()

,elem_1,elem_2,similarity
0,joke_1,joke_2,0.202696
1,joke_1,joke_3,0.173774
2,joke_1,joke_4,0.018405
3,joke_1,joke_5,0.006122
4,joke_1,joke_6,-0.086289


In [8]:
def recommendation_2(active_user_id, no_normalized_ratings_df, similarity_matrix):
    # si prende la riga relativa all'utente con l'id specificato
    active_user = no_normalized_ratings_df[no_normalized_ratings_df['user_id'] == active_user_id]
    # si considerano solo i rating espressi dall'utente target
    active_user_rating = active_user.iloc[:, 2:]
    # si prendono le barzellette raccomandabili all'utente
    recommendation_columns = []
    for column in active_user_rating.columns:
        if math.isnan(active_user_rating[column].values[0]):
            recommendation_columns.append(column)

    # si prendono le barzellette già valutate dall'utente
    rated_items = list(set(list(active_user_rating.columns)) - set(recommendation_columns))
    # si fa la raccomandazione
    top_score = -np.inf
    joke_to_suggest = ''
    final = []
    for joke in recommendation_columns:
        score = score_user_item(joke, rated_items, active_user_rating, similarity_matrix)
        print("per la barzelletta " + joke + " si è predetto un punteggio di: " + str(score))
        if score > top_score:
            top_score = score
            joke_to_suggest = joke
        app_2 = {}
        app_2["user_id"] = active_user_id
        app_2[joke] = score
        final.append(app_2)
    print('\nil valore più alto predetto è ' + str(top_score) + ' su 10')
    print('quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è: ', joke_to_suggest)
    return final
    

In [9]:
recommendation_2(1001, no_normalized_ratings_df, similarity_matrix)

per la barzelletta joke_1 si è predetto un punteggio di: 5.489189111538942
per la barzelletta joke_2 si è predetto un punteggio di: 5.87489688531074
per la barzelletta joke_3 si è predetto un punteggio di: 5.530364213231963
per la barzelletta joke_4 si è predetto un punteggio di: 5.683020665121501
per la barzelletta joke_9 si è predetto un punteggio di: 5.629541237464796
per la barzelletta joke_23 si è predetto un punteggio di: 5.5809204940367145
per la barzelletta joke_24 si è predetto un punteggio di: 5.7830860611316695
per la barzelletta joke_33 si è predetto un punteggio di: 5.6816758536638385
per la barzelletta joke_37 si è predetto un punteggio di: 5.920645912010233
per la barzelletta joke_43 si è predetto un punteggio di: 5.719735411098295
per la barzelletta joke_47 si è predetto un punteggio di: 8.071662854189437
per la barzelletta joke_55 si è predetto un punteggio di: 5.573499141349482
per la barzelletta joke_57 si è predetto un punteggio di: 5.923930500445841
per la barzelle

[{'user_id': 1001, 'joke_1': 5.489189111538942},
 {'user_id': 1001, 'joke_2': 5.87489688531074},
 {'user_id': 1001, 'joke_3': 5.530364213231963},
 {'user_id': 1001, 'joke_4': 5.683020665121501},
 {'user_id': 1001, 'joke_9': 5.629541237464796},
 {'user_id': 1001, 'joke_23': 5.5809204940367145},
 {'user_id': 1001, 'joke_24': 5.7830860611316695},
 {'user_id': 1001, 'joke_33': 5.6816758536638385},
 {'user_id': 1001, 'joke_37': 5.920645912010233},
 {'user_id': 1001, 'joke_43': 5.719735411098295},
 {'user_id': 1001, 'joke_47': 8.071662854189437},
 {'user_id': 1001, 'joke_55': 5.573499141349482},
 {'user_id': 1001, 'joke_57': 5.923930500445841},
 {'user_id': 1001, 'joke_58': 5.9793172715780365},
 {'user_id': 1001, 'joke_60': 5.916319952555864},
 {'user_id': 1001, 'joke_64': 5.975977616100421},
 {'user_id': 1001, 'joke_67': 5.726215401671424},
 {'user_id': 1001, 'joke_71': 5.473898526952023},
 {'user_id': 1001, 'joke_72': 9.827958290733308},
 {'user_id': 1001, 'joke_73': 4.587967997499566},
 {